在沙盒环境，多特征非线性合成及进行组合优化

In [1]:
import os,pdb,itertools,copy,datetime
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import pandas as pd
import numpy as np
from ultron.factor.data.processing import factor_processing
from ultron.factor.data.standardize import standardize
from ultron.optimize.model.treemodel import RandomForestRegressor
from ultron.optimize.model.linearmodel import LinearRegression
from ultron.strategy.experimental.single_factor import SingleFactor
from ultron.strategy.fut_strategy import create_params
from ultron.strategy.strategy import StrategyEngine
from ultron.kdutils.parallel import delayed, Parallel

/var/log/ultron/2022-09-27.log


In [3]:
from ultron.env import *

In [4]:
enable_example_env()

2022-09-27 19:08:30,236 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


#### 加载行情数据

In [5]:
market_data = pd.read_csv(os.path.join(g_project_data, 'market_data.csv'), index_col=0)
market_data['trade_date'] = pd.to_datetime(market_data['trade_date'])
market_data.head()

,trade_date,code,openPrice,highestPrice,lowestPrice,closePrice,turnoverVol
0,2017-10-27,A,4462.578191,4463.801485,4413.646412,4435.665713,158774
1,2017-10-27,AL,15625.658581,15658.904663,15430.931529,15449.929290,293630
2,2017-10-27,BU,3310.339921,3336.950371,3283.729472,3302.356787,461826
3,2017-10-27,C,2009.751001,2014.561895,2001.331936,2002.534660,375480
4,2017-10-27,CF,20517.496003,20531.174333,20408.069357,20449.104349,84032


#### 选择中因子

In [6]:
sel_factor = pd.read_csv(os.path.join(g_project_data, 'sel_factor.csv'), index_col=0)
sel_factor = sel_factor.drop_duplicates(subset=['factor','window'])
sel_factor = sel_factor[(sel_factor['factor'] != 'inventory')]
sel_factor = sel_factor[(sel_factor['factor'] != 'profitratio')]
sel_factor.head()

,factor,window,weekday,bins
0,BM_MainFar_80D,23,5,5
1,BM_MainFar_80D,25,5,5
2,BM_MainFar_80D,27,5,5
3,BM_RecentFar_20D,5,1,5
4,BM_RecentFar_40D,3,1,3


#### 读取因子

In [7]:
total_data = pd.read_csv(os.path.join(g_project_data, 'factor.csv'), index_col=0)
total_data['trade_date'] = pd.to_datetime(total_data['trade_date'] )
factor_data = total_data[['trade_date','code'] + sel_factor['factor'].unique().tolist()]
factor_data.head()

,trade_date,code,BM_MainFar_80D,BM_RecentFar_20D,BM_RecentFar_40D,BM_RecentFar_80D,BM_RecentSecond_20D,BM_RecentSecond_40D,B_FarSpot,B_MainSpot,...,R_UpVolatility_1_40D,R_UpVolatility_1_60D,TS_MainFar,TS_RecentFar,TS_RecentSecond,T_DnIntraday_5D,T_DnVolatility_1_10D,T_DnVolatility_2_20D,WeightNetIntTotalChg5D,WeightShortVolRelTotIntChg
0,2017-10-27,A,-0.033259,-0.026646,-0.019436,-0.041974,-0.023047,-0.013509,-0.042729,0.002378,...,-0.003930,-0.005081,-0.057610,-0.079619,-0.104757,-0.008350,-0.007715,-0.002168,-0.000633,-0.037579
1,2017-10-27,AL,-0.001423,0.001697,-0.000937,0.000587,0.001133,-0.000539,-0.076121,-0.084726,...,-0.010231,-0.012803,-0.069381,-0.068413,-0.067663,-0.005843,-0.008381,0.000165,-0.000352,-0.012891
2,2017-10-27,BU,-0.016537,0.059635,-0.032271,-0.034618,0.069999,-0.027086,-0.124574,-0.321128,...,-0.010399,-0.012054,-0.102761,-0.124225,-0.159247,-0.005098,-0.009538,0.001268,0.002481,0.275875
3,2017-10-27,C,0.007939,-0.005224,-0.014003,0.025361,-0.001541,-0.007955,-0.006522,0.172635,...,-0.004324,-0.004901,-0.063351,-0.072537,-0.092543,-0.002866,-0.003349,-0.000979,0.002547,0.245555
4,2017-10-27,CF,-0.023960,0.002346,-0.028774,-0.008043,0.004838,-0.009858,0.056640,0.311288,...,-0.006368,-0.006944,-0.025782,-0.024471,-0.003359,-0.003975,-0.004373,-0.001420,-0.000643,-0.131799


#### 加载行业信息

In [8]:
industry_data = pd.read_csv(os.path.join(g_project_data, 'industry_data.csv'), index_col=0)
industry_data.head()

,code,industry
0,RB,black_series
1,I,black_series
2,ZC,black_series
3,JM,black_series
4,HC,black_series


#### 时序标准化

In [9]:
parallel = Parallel(n_jobs=4, verbose=1, pre_dispatch='2*n_jobs')

In [10]:
def _build(total_data, window, columns):
    sf = SingleFactor(factor_data=None, market_data=None, codes=None, columns=None)
    normalize_data = sf.normalize(factor_data=total_data.dropna(subset=columns), 
                                    windows=window, columns=columns)
    normalize_data = normalize_data.sort_values(by=['trade_date','code'])
    return normalize_data.set_index(['trade_date','code']).rename(
        columns={columns[0]:columns[0] + '_' + str(window)})

In [11]:
out = parallel(delayed(_build)(factor_data[['trade_date','code', v['factor']]],
                   window=v['window'], columns=[v['factor']]) for v in sel_factor.to_dict(orient='records'))

In [12]:
factors_data = pd.concat(out, axis=1).reset_index().fillna(0)
factors_data.head()

,trade_date,code,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,...,TS_MainFar_5,TS_RecentFar_7,TS_RecentSecond_7,T_DnIntraday_5D_23,T_DnVolatility_1_10D_21,T_DnVolatility_2_20D_25,T_DnVolatility_2_20D_27,WeightNetIntTotalChg5D_11,WeightNetIntTotalChg5D_13,WeightShortVolRelTotIntChg_9
0,2017-10-31,A,0.0,0.0,0.0,0.0,-0.397406,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-10-31,AL,0.0,0.0,0.0,0.0,0.429146,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-10-31,BU,0.0,0.0,0.0,0.0,1.145945,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-10-31,C,0.0,0.0,0.0,0.0,1.127665,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-10-31,CF,0.0,0.0,0.0,0.0,1.040272,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 横截面标准化

期货品种不做去极值和中性化

In [13]:
diff_cols = ['trade_date','code']
features = [col for col in factors_data.columns if col not in diff_cols]

In [14]:
alpha_res = []
grouped = factors_data.groupby(['trade_date'])
for k, g in grouped:
    new_factors = factor_processing(g[features].values,
                 pre_process=[standardize])
    f = pd.DataFrame(new_factors, columns=features)
    for k in diff_cols:
        f[k] = g[k].values
    alpha_res.append(f)

/home/kerry/work/workenv/ultron/lib/python3.6/site-packages/Finance_Ultron-0.8.6-py3.6-linux-x86_64.egg/ultron/factor/data/standardize.py:22: RuntimeWarning: invalid value encountered in subtract
  return (x - simple_mean(x, axis=0)) / np.maximum(simple_std(x, axis=0, ddof=ddof), 1e-8)


In [15]:
factors_data = pd.concat(alpha_res)
factors_data.tail()

,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,BM_RecentFar_80D_9,BM_RecentFar_80D_11,...,TS_RecentSecond_7,T_DnIntraday_5D_23,T_DnVolatility_1_10D_21,T_DnVolatility_2_20D_25,T_DnVolatility_2_20D_27,WeightNetIntTotalChg5D_11,WeightNetIntTotalChg5D_13,WeightShortVolRelTotIntChg_9,trade_date,code
30,-1.476446,-1.582866,-1.660658,-1.378918,-0.761049,-1.491990,-1.548249,-1.322412,-1.561855,-1.494890,...,0.792496,0.279589,-0.568689,-0.184132,-0.234938,-0.122780,-0.360587,-0.252067,2022-06-22,TA
31,-0.087925,-0.140685,-0.174119,0.737069,-1.077857,-0.444036,-0.620455,-0.432019,-0.429449,-0.466354,...,0.458422,-0.978035,-1.142478,-0.939997,-0.987123,0.607201,0.203519,-0.531509,2022-06-22,V
32,-1.147232,-0.968060,-0.922699,-1.510556,-1.203488,-1.447780,-1.385261,-1.424192,-1.471564,-1.515301,...,-2.133296,-1.419082,-1.314467,-1.868960,-1.925835,0.901514,0.922691,0.496849,2022-06-22,Y
33,0.206059,0.167452,0.139033,0.447884,-0.253253,-0.608452,-0.519678,-0.489815,-1.865444,-1.926757,...,-0.999571,2.374130,0.355085,-0.581136,-0.640288,-1.046225,-0.186613,0.219181,2022-06-22,ZC
34,1.757122,1.660916,1.357403,0.818272,-0.948261,0.920428,0.805499,0.957197,0.541772,0.589358,...,0.057476,-0.667581,0.415297,1.773770,1.807674,-0.747155,-0.579080,0.876367,2022-06-22,ZN


构建模型X,Y值

##### 目前使用收益率为第二天的收益率为Y，来构建模型

In [16]:
def returns(market_data):
    price_tb = market_data['closePrice'].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb.shift(-1) / price_tb)
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(market_data.index)
    return_tb.name = 'nxt1_ret'
    return return_tb

In [17]:
rets = returns(market_data.set_index(['trade_date','code']))
rets.head()

trade_date  code
2017-10-27  A      -0.000276
            AL      0.001222
            BU      0.019945
            C      -0.005420
            CF      0.001003
Name: nxt1_ret, dtype: float64

In [18]:
train_data = factors_data.merge(rets, on=['trade_date','code'])
train_data = train_data.dropna(subset=['nxt1_ret']).fillna(0)
train_data.head()

,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,BM_RecentFar_80D_9,BM_RecentFar_80D_11,...,T_DnIntraday_5D_23,T_DnVolatility_1_10D_21,T_DnVolatility_2_20D_25,T_DnVolatility_2_20D_27,WeightNetIntTotalChg5D_11,WeightNetIntTotalChg5D_13,WeightShortVolRelTotIntChg_9,trade_date,code,nxt1_ret
0,0.0,0.0,0.0,0.0,-0.331122,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-31,A,-0.000275
1,0.0,0.0,0.0,0.0,0.564609,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-31,AL,-0.007976
2,0.0,0.0,0.0,0.0,1.341401,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-31,BU,-0.011236
3,0.0,0.0,0.0,0.0,1.321591,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-31,C,0.002411
4,0.0,0.0,0.0,0.0,1.226883,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-31,CF,0.000334


可使用多个机器学习模型，这里为了执行效率，使用线性回归模型

In [19]:
#model = RandomForestRegressor(features=features)
model = LinearRegression(features=features, fit_intercept=False)

In [20]:
%%time
model.fit(train_data, train_data['nxt1_ret'].values)

CPU times: user 192 ms, sys: 28 ms, total: 220 ms
Wall time: 94.4 ms


#### 组合构建

此处暂时使用等权为初始权重

In [21]:
def _weighted(data, equal=1):
    data['factor'] =1
    weighted =  1 / len(data)
    weighted = pd.DataFrame([weighted for i in range(0, len(data))],
                                columns=['factor'],
                                index=data.index)
    weighted['code'] = data['code'].values
    weighted = weighted.reset_index().drop(['trade_date'],axis=1)
    return weighted.rename(columns={'factor':'weight'})
benchmark_weighed = factors_data[['trade_date','code']].set_index('trade_date').groupby(
    level=['trade_date']).apply(
    lambda x: _weighted(x)).reset_index().drop(['level_1'],axis=1)
benchmark_weighed.head()

,trade_date,weight,code
0,2017-10-31,0.032258,A
1,2017-10-31,0.032258,AL
2,2017-10-31,0.032258,BU
3,2017-10-31,0.032258,C
4,2017-10-31,0.032258,CF


In [22]:
industry_dummy = pd.get_dummies(industry_data.set_index('code')['industry']).reset_index()
industry_dummy.head()

,code,agricultural,black_series,chemicals,non_ferrous
0,RB,0,1,0,0
1,I,0,1,0,0
2,ZC,0,1,0,0
3,JM,0,1,0,0
4,HC,0,1,0,0


In [23]:
factors_data = factors_data.merge(
    industry_data, on=['code']).merge(
    industry_dummy, on=['code']).set_index(['trade_date','code'])
factors_data = factors_data.reset_index()
factors_data.tail()

,trade_date,code,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,...,T_DnVolatility_2_20D_25,T_DnVolatility_2_20D_27,WeightNetIntTotalChg5D_11,WeightNetIntTotalChg5D_13,WeightShortVolRelTotIntChg_9,industry,agricultural,black_series,chemicals,non_ferrous
37176,2022-06-16,LH,-1.112855,-1.037425,-1.138118,-1.166361,-1.239084,-1.528686,-1.456629,-1.300855,...,-0.286847,-0.399211,-0.923560,-0.979208,-1.036734,agricultural,1,0,0,0
37177,2022-06-17,LH,1.306678,1.050566,0.872997,0.099553,-0.741284,-1.539429,-1.545584,-1.752069,...,-0.358204,-0.483033,0.518428,0.422075,0.511451,agricultural,1,0,0,0
37178,2022-06-20,LH,1.405912,1.325234,1.102284,-0.978639,-1.720337,-1.749304,-1.781356,-1.827989,...,0.524848,0.390461,0.773097,0.659696,0.504438,agricultural,1,0,0,0
37179,2022-06-21,LH,1.694437,1.690552,1.499223,-0.400917,0.316539,-1.205826,-1.112083,-1.270934,...,0.699334,0.590639,1.220513,1.039573,0.671012,agricultural,1,0,0,0
37180,2022-06-22,LH,1.083093,1.141599,1.097464,0.076741,1.196897,-0.690854,-0.532233,-0.708458,...,0.679594,0.607673,0.749293,0.652983,0.638815,agricultural,1,0,0,0


In [24]:
neutralized_styles = factors_data['industry'].unique().tolist()

In [25]:
factors_data = factors_data.merge(
    benchmark_weighed, on=['trade_date','code']).sort_values(
    by=['trade_date','code'],ascending=True
)

In [26]:
factors_data = factors_data.rename(columns={'industry':'industry_code'})
is_in_benchmark = (factors_data.weight != 0.).astype(float).values.reshape((-1, 1))
factors_data.loc[:, 'benchmark'] = is_in_benchmark
factors_data.loc[:, 'total'] = np.ones_like(is_in_benchmark)
factors_data.tail()

,trade_date,code,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,...,WeightNetIntTotalChg5D_13,WeightShortVolRelTotIntChg_9,industry_code,agricultural,black_series,chemicals,non_ferrous,weight,benchmark,total
30442,2022-06-22,TA,-1.476446,-1.582866,-1.660658,-1.378918,-0.761049,-1.491990,-1.548249,-1.322412,...,-0.360587,-0.252067,chemicals,0,0,1,0,0.028571,1.0,1.0
31570,2022-06-22,V,-0.087925,-0.140685,-0.174119,0.737069,-1.077857,-0.444036,-0.620455,-0.432019,...,0.203519,-0.531509,chemicals,0,0,1,0,0.028571,1.0,1.0
32698,2022-06-22,Y,-1.147232,-0.968060,-0.922699,-1.510556,-1.203488,-1.447780,-1.385261,-1.424192,...,0.922691,0.496849,agricultural,1,0,0,0,0.028571,1.0,1.0
33826,2022-06-22,ZC,0.206059,0.167452,0.139033,0.447884,-0.253253,-0.608452,-0.519678,-0.489815,...,-0.186613,0.219181,black_series,0,1,0,0,0.028571,1.0,1.0
34954,2022-06-22,ZN,1.757122,1.660916,1.357403,0.818272,-0.948261,0.920428,0.805499,0.957197,...,-0.579080,0.876367,non_ferrous,0,0,0,1,0.028571,1.0,1.0


In [27]:
factors_data = factors_data.merge(rets.reset_index(), on=['trade_date','code']).rename(
    columns={'nxt1_ret':'pev1_ret'})
factors_data.head()

,trade_date,code,BM_MainFar_80D_23,BM_MainFar_80D_25,BM_MainFar_80D_27,BM_RecentFar_20D_5,BM_RecentFar_40D_3,BM_RecentFar_40D_11,BM_RecentFar_40D_9,BM_RecentFar_40D_13,...,WeightShortVolRelTotIntChg_9,industry_code,agricultural,black_series,chemicals,non_ferrous,weight,benchmark,total,pev1_ret
0,2017-10-31,A,0.0,0.0,0.0,0.0,-0.331122,0.0,0.0,0.0,...,0.0,agricultural,1,0,0,0,0.032258,1.0,1.0,-0.000275
1,2017-10-31,AL,0.0,0.0,0.0,0.0,0.564609,0.0,0.0,0.0,...,0.0,non_ferrous,0,0,0,1,0.032258,1.0,1.0,-0.007976
2,2017-10-31,BU,0.0,0.0,0.0,0.0,1.341401,0.0,0.0,0.0,...,0.0,chemicals,0,0,1,0,0.032258,1.0,1.0,-0.011236
3,2017-10-31,C,0.0,0.0,0.0,0.0,1.321591,0.0,0.0,0.0,...,0.0,agricultural,1,0,0,0,0.032258,1.0,1.0,0.002411
4,2017-10-31,CF,0.0,0.0,0.0,0.0,1.226883,0.0,0.0,0.0,...,0.0,agricultural,1,0,0,0,0.032258,1.0,1.0,0.000334


In [28]:
effective_codes = factors_data.industry_code.unique().tolist() ### 有效行业
invalid_codes = [] ### 无效行业
target_vol = 0.05 ### 目标波动率
turn_over_target = 0.4  ### 换手率
lbound = -0.1 ### 单个标的优化下限
ubound = 0.1 ### 单个标的优化上限
benchmark_lower = -1.00 ## 基于基准下限
benchmark_upper = 1.001 ## 基于基准上限
total_lower = -0.01 ## 组合下限
total_upper = 0.01 ## 组合上限
effective_industry_lower = -0.1 ## 有效行业下限
effective_industry_upper = 0.1 ## 有效行业上限
invalid_industry_lower = -0.2 ## 无效行业下限
invalid_industry_lower = 0.2 ## 无效行业上限
cov_method = 'unshrunk' ## 协方差收益率模式
method = 'long_short'

In [29]:
params = create_params(
    effective_codes=effective_codes, invalid_codes=invalid_codes,
    target_vol=target_vol,turn_over_target=turn_over_target, 
    lbound=lbound, ubound=ubound,
    benchmark_lower=benchmark_lower,benchmark_upper=benchmark_upper,
    total_lower=total_lower,total_upper=total_upper,
    effective_industry_lower=effective_industry_lower,
    effective_industry_upper=effective_industry_upper,
    method=method,
    is_benchmark=1)

#### 构建模型

In [30]:
models = {}
factors_data_groups = factors_data.groupby('trade_date')
for ref_date, _ in factors_data_groups:
    models[ref_date] = model

In [31]:
strategy = StrategyEngine.create_class(name='fut')(alpha_model=models, total_data=factors_data, 
                    start_date=datetime.datetime(2021,1,4))

In [32]:
positions = strategy.create_positions(params=params)

2022-09-27 19:09:09,471 - [fut_strategy.py:312] - ultron - INFO - starting re-balance ...
2022-09-27 19:09:09,489 - [fut_strategy.py:330] - ultron - INFO - running setting finished ...
2022-09-27 19:09:10,348 - [fut_strategy.py:373] - ultron - INFO - 2021-01-04 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,362 - [fut_strategy.py:373] - ultron - INFO - 2021-01-05 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,373 - [fut_strategy.py:373] - ultron - INFO - 2021-01-06 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,384 - [fut_strategy.py:373] - ultron - INFO - 2021-01-07 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,394 - [fut_strategy.py:373] - ultron - INFO - 2021-01-08 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,405 - [fut_strategy.py:373] - ultron - INFO - 2021-01-11 00:00:00 re-balance: 34 codes
2022-09-27 19:09:10,417 - [fut_strategy.py:373] - ultron - INFO - 2021-01-12 00:00:00 re-balance: 35 codes
2022-09-27 19:09:10,428 - [fut_strategy.py:373] - ultron - INFO - 

2022-09-27 19:09:11,166 - [fut_strategy.py:373] - ultron - INFO - 2021-04-27 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,177 - [fut_strategy.py:373] - ultron - INFO - 2021-04-28 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,187 - [fut_strategy.py:373] - ultron - INFO - 2021-04-29 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,198 - [fut_strategy.py:373] - ultron - INFO - 2021-04-30 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,209 - [fut_strategy.py:373] - ultron - INFO - 2021-05-06 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,220 - [fut_strategy.py:373] - ultron - INFO - 2021-05-07 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,231 - [fut_strategy.py:373] - ultron - INFO - 2021-05-10 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,242 - [fut_strategy.py:373] - ultron - INFO - 2021-05-11 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,253 - [fut_strategy.py:373] - ultron - INFO - 2021-05-12 00:00:00 re-balance: 35 codes
2022-09-27 19:09:11,264 - [fut_strate

2022-09-27 19:09:12,006 - [fut_strategy.py:373] - ultron - INFO - 2021-08-18 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,017 - [fut_strategy.py:373] - ultron - INFO - 2021-08-19 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,028 - [fut_strategy.py:373] - ultron - INFO - 2021-08-20 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,039 - [fut_strategy.py:373] - ultron - INFO - 2021-08-23 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,050 - [fut_strategy.py:373] - ultron - INFO - 2021-08-24 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,061 - [fut_strategy.py:373] - ultron - INFO - 2021-08-25 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,072 - [fut_strategy.py:373] - ultron - INFO - 2021-08-26 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,083 - [fut_strategy.py:373] - ultron - INFO - 2021-08-27 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,094 - [fut_strategy.py:373] - ultron - INFO - 2021-08-30 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,104 - [fut_strate

2022-09-27 19:09:12,860 - [fut_strategy.py:373] - ultron - INFO - 2021-12-14 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,871 - [fut_strategy.py:373] - ultron - INFO - 2021-12-15 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,882 - [fut_strategy.py:373] - ultron - INFO - 2021-12-16 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,893 - [fut_strategy.py:373] - ultron - INFO - 2021-12-17 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,904 - [fut_strategy.py:373] - ultron - INFO - 2021-12-20 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,915 - [fut_strategy.py:373] - ultron - INFO - 2021-12-21 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,927 - [fut_strategy.py:373] - ultron - INFO - 2021-12-22 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,938 - [fut_strategy.py:373] - ultron - INFO - 2021-12-23 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,949 - [fut_strategy.py:373] - ultron - INFO - 2021-12-24 00:00:00 re-balance: 35 codes
2022-09-27 19:09:12,960 - [fut_strate

2022-09-27 19:09:13,725 - [fut_strategy.py:373] - ultron - INFO - 2022-04-12 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,737 - [fut_strategy.py:373] - ultron - INFO - 2022-04-13 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,748 - [fut_strategy.py:373] - ultron - INFO - 2022-04-14 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,759 - [fut_strategy.py:373] - ultron - INFO - 2022-04-15 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,771 - [fut_strategy.py:373] - ultron - INFO - 2022-04-18 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,782 - [fut_strategy.py:373] - ultron - INFO - 2022-04-19 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,793 - [fut_strategy.py:373] - ultron - INFO - 2022-04-20 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,804 - [fut_strategy.py:373] - ultron - INFO - 2022-04-21 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,816 - [fut_strategy.py:373] - ultron - INFO - 2022-04-22 00:00:00 re-balance: 35 codes
2022-09-27 19:09:13,827 - [fut_strate